In [ ]:
# Setup environment
# !apt-get -qq install xxd
# !pip3 install pandas numpy matplotlib
# !pip3 install tensorflow==2.13.0


With model '''
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(100, activation='relu')) # relu is used for performance
model.add(tf.keras.layers.Dense(50, activation='relu'))
model.add(tf.keras.layers.Dense(NUM_GESTURES, activation='softmax')) # softmax is used, because we only expect one gesture to occur per input
model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
history = model.fit(inputs_train, outputs_train, epochs=200, batch_size=1, validation_data=(inputs_validate, outputs_validate))
'''

result:<br>
tests by letter:  [24 21 27 27 17 25 34 24 29 22 26 35 22 35 17 27 28 25 23 18 21 22 30 25 32 31]<br>
wrong predict by letter:  [ 5  3  4  4  0  6 13  0  1  2  2  6  2 12  3  5  6  7  5  4  3  2  7 11 12  5]<br>
wrong rate:  [0.21 0.14 0.15 0.15 0.   0.24 0.38 0.   0.03 0.09 0.08 0.17 0.09 0.34
 0.18 0.19 0.21 0.28 0.22 0.22 0.14 0.09 0.23 0.44 0.38 0.16]<br>


add another layer<br>
tests by letter:  [24 21 27 27 17 25 34 24 29 22 26 35 22 35 17 27 28 25 23 18 21 22 30 25
 32 31]<br>
wrong predict by letter:  [ 7  6  9  2  0  5 14 11 10  8  1 20  6 17  7 13  2  7  3  6  6  4  8  4
 17  3]<br>
wrong rate:  [0.29 0.29 0.33 0.07 0.   0.2  0.41 0.46 0.34 0.36 0.04 0.57 0.27 0.49
 0.41 0.48 0.07 0.28 0.13 0.33 0.29 0.18 0.27 0.16 0.53 0.1 ]<br>
 r2 score 0.73 <br>

 increase epoches to 500, r2 score 0.55, error start increast around 150-200, <br>

 use two layer, first layer 120 tensors, second 60 tensors. <br>
 tests by letter:  [24 21 27 27 17 25 34 24 29 22 26 35 22 35 17 27 28 25 23 18 21 22 30 25
 32 31] <br>
wrong predict by letter:  [ 2  5  4  2  0  2  6 16  3  2  4  7 10 11  4  3  4  7  0  2  5  1 10  6
 10  4] <br>
wrong rate:  [0.08 0.24 0.15 0.07 0.   0.08 0.18 0.67 0.1  0.09 0.15 0.2  0.45 0.31
 0.24 0.11  0.14 0.28 0.   0.11 0.24 0.05 0.33 0.24 0.31 0.13] <br>

 r2 score 0.82, Model is 383556 bytes <br>
 Sketch uses 766672 bytes (94%) of program storage space. Maximum is 811008 bytes.<br>
Global variables use 59936 bytes (25%) of dynamic memory, leaving 177632 bytes for local variables. Maximum is 237568 bytes. <br>

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import glob
import random

file_path = "processed_data/*"

print(f"TensorFlow version = {tf.__version__}\n")

# Set a fixed random seed value, for reproducibility, this will allow us to get
# the same random numbers each time the notebook is run
SEED = 1337
np.random.seed(SEED)
tf.random.set_seed(SEED)

# the list of gestures that data is available for
GESTURES = "abcdefghijklmnopqrstuvwxyz"

SAMPLES_PER_GESTURE = 120

NUM_GESTURES = 26

# create a one-hot encoded matrix that is used in the output
ONE_HOT_ENCODED_GESTURES = np.eye(NUM_GESTURES)

inputs = []
outputs = []

datafiles = glob.glob(file_path)
# datafiles.sort()
print("total files", len(datafiles))

for datafile in datafiles:
    out = []    

    tensor = pd.read_csv(datafile)
    inputs.append(np.array(tensor.values.ravel()))

    gesture_index=0
    for i in range(NUM_GESTURES):
        if datafile[15] == GESTURES[i]:
            gesture_index = i
    output = ONE_HOT_ENCODED_GESTURES[gesture_index]
    outputs.append(output)
    # print ("processed ", datafile, "output=", GESTURES[gesture_index])
    
# convert the list to numpy array
inputs = np.array(inputs)
outputs = np.array(outputs)

print ("input shape: ", inputs.shape, " output shape", outputs.shape)

print("Data set parsing and preparation complete.")

In [ ]:
# Randomize the order of the inputs, so they can be evenly distributed for training, testing, and validation
# https://stackoverflow.com/a/37710486/2020087
num_inputs = len(inputs)
randomize = np.arange(num_inputs)
np.random.shuffle(randomize)

# Swap the consecutive indexes (0, 1, 2, etc) with the randomized indexes
inputs = inputs[randomize]
outputs = outputs[randomize]

# Split the recordings (group of samples) into three sets: training, testing and validation
TRAIN_SPLIT = int(0.6 * num_inputs)
TEST_SPLIT = int(0.2 * num_inputs + TRAIN_SPLIT)

inputs_train, inputs_test, inputs_validate = np.split(inputs, [TRAIN_SPLIT, TEST_SPLIT])
outputs_train, outputs_test, outputs_validate = np.split(outputs, [TRAIN_SPLIT, TEST_SPLIT])

print("Data set randomization and splitting complete.")

In [ ]:
# build the model and train it
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(120, activation='relu')) # relu is used for performance
# model.add(tf.keras.layers.Dense(50, activation='relu'))
model.add(tf.keras.layers.Dense(60, activation='relu'))
model.add(tf.keras.layers.Dense(NUM_GESTURES, activation='softmax')) # softmax is used, because we only expect one gesture to occur per input
model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
history = model.fit(inputs_train, outputs_train, epochs=100, batch_size=1, validation_data=(inputs_validate, outputs_validate))



In [ ]:
# increase the size of the graphs. The default size is (6,4).
plt.rcParams["figure.figsize"] = (5,3)

# graph the loss, the model above is configure to use "mean squared error" as the loss function
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'g.', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

print(plt.rcParams["figure.figsize"])

In [ ]:
# graph the loss again skipping a bit of the start
SKIP = 100
plt.plot(epochs[SKIP:], loss[SKIP:], 'g.', label='Training loss')
plt.plot(epochs[SKIP:], val_loss[SKIP:], 'b.', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# graph of mean absolute error
mae = history.history['mae']
val_mae = history.history['val_mae']
plt.plot(epochs[SKIP:], mae[SKIP:], 'g.', label='Training MAE')
plt.plot(epochs[SKIP:], val_mae[SKIP:], 'b.', label='Validation MAE')
plt.title('Training and validation mean absolute error')
plt.xlabel('Epochs')
plt.ylabel('MAE')
plt.legend()
plt.show()


In [ ]:
print("input shape" , inputs_test.shape)
print(type(inputs_test))
# print (len(data1))
# inputs_test = np.concatenate((inputs_test, [np.array(data1)]), axis=0)
# print("input shape" , inputs_test.shape)

In [ ]:
t1 = pd.read_csv("processed_data/a_1.dat")
# print(t1)
single_sample = np.array(t1.values.ravel())
# single_sample1 = (t1.melt().value.tolist())
print(type(single_sample))
prediction = model.predict(np.expand_dims(single_sample, axis=0))
# prediction1 = model.predict(np.expand_dims(single_sample1, axis=0))

# formatted_numbers = ["{:.2f}".format(number) for number in prediction[0]]

# print("Formatted Numbers:", formatted_numbers)
# print("Prediction:", prediction)

print("predictions =\n", np.round(prediction, decimals=3))
# print(single_sample)
# print(single_sample1)

In [ ]:
# t1 = pd.read_csv("processed_data/a_1.dat")
# [np.array(t1.values.ravel())]

# inputs_test = np.concatenate((inputs_test, [np.array(t1.values.ravel())]), axis=0)
# # 

In [ ]:
# use the model to predict the test inputs
predictions = model.predict(inputs_test)
print("pred shape" , predictions.shape)
# predictions = model.predict(inputs_test[0].reshape(1,1248))
# print(predictions[0])
# print(outputs_test[0])
# print (inputs_test[0])

# pd.DataFrame(inputs_test[0]).to_csv("test1.h", sep=',', encoding='utf-8', index=False, header=False)
# !echo "const unsigned char tt[] = {" > ./tt.h
# !cat "test1.csv" | xxd -i      >> ./tt.h
# !echo "};"                              >> ./tt.h

# print the predictions and the expected ouputs
print("predictions =\n", np.round(predictions, decimals=3))

print("actual =\n", outputs_test)
a = np.round(predictions - outputs_test, decimals=0)
print(a)
# Plot the predictions along with to the test data
# plt.clf()
# plt.title('Training data predicted vs actual values')
# plt.plot( outputs_test, 'b.', label='Actual')
# plt.plot( predictions, 'r.', label='Predicted')
# plt.show()


In [ ]:
print(outputs_test[:, 0].dtype)
print(predictions[:, 0].dtype)

In [ ]:
pre = (predictions + 0.5).astype(int)
# pre
a = pre-outputs_test
tests_by_letter = outputs_test.astype(bool).sum(axis=0)
print("tests by letter: ", tests_by_letter)
wrong_predict_by_letter = a.astype(bool).sum(axis=0)
print("wrong predict by letter: ", wrong_predict_by_letter)
print("wrong rate: ", np.round(wrong_predict_by_letter/tests_by_letter, decimals=2))

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
# accuracy = accuracy_score(outputs_test, predictions)
# precision = precision_score(outputs_test, predictions)
# recall = recall_score(outputs_test, predictions)
# f1 = f1_score(outputs_test, predictions)
# conf_matrix = confusion_matrix(outputs_test, predictions)

# print(f"Accuracy: {accuracy}")
# print(f"Precision: {precision}")
# print(f"Recall: {recall}")
# print(f"F1-Score: {f1}")
# print(f"Confusion Matrix:\n{conf_matrix}")
# Initialize an array to store accuracy for each class

# class_accuracies = []

# # Iterate over each class (assuming axis 1 represents classes)
# for class_index in range(26):
#     true_labels_class = outputs_test[:, class_index]
#     predicted_labels_class = pre[:, class_index]
#     accuracy = accuracy_score(true_labels_class, predicted_labels_class)
#     class_accuracies.append(accuracy)

# # Calculate micro-average accuracy (overall accuracy)
# micro_average_accuracy = accuracy_score(outputs_test, predictions)

# # Calculate macro-average accuracy (average accuracy across classes)
# macro_average_accuracy = sum(class_accuracies) / len(class_accuracies)

# # Print individual class accuracies and the macro/micro averages
# for class_index, accuracy in enumerate(class_accuracies):
#     print(f"Class {class_index}: Accuracy = {accuracy}")

# print(f"Micro-average accuracy: {micro_average_accuracy}")
# print(f"Macro-average accuracy: {macro_average_accuracy}")
# In the code above:

# We iterate over each class, treating it as a binary classification problem by selecting the true labels and predicted labels for that class.
# We calculate the accuracy for each class separately and store it in the class_accuracies list.
# We compute the micro-average accuracy, which is the overall accuracy across all samples and classes.
# We compute the macro-average accuracy, which is the average accuracy across all classes.
# This approach allows you to evaluate the performance of your multi-class classification model for each individual class and provides overall accuracy metrics as well.







In [ ]:
# import matplotlib.pyplot as plt

# plt.scatter(outputs_test, predictions)
# plt.xlabel("True Values")
# plt.ylabel("Predictions")
# plt.show()

from sklearn.metrics import r2_score


r2 = r2_score(outputs_test, predictions)
print("r2=", r2)

In [ ]:
outputs_test[0]

In [ ]:
(predictions[0]+0.5).astype(int)

In [ ]:
# tf.saved_model.save(model, 'my_saved_model')

In [ ]:
# Convert the model to the TensorFlow Lite format without quantization
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model to disk
open("gesture_model.tflite", "wb").write(tflite_model)
  
import os
basic_model_size = os.path.getsize("gesture_model.tflite")
print("Model is %d bytes" % basic_model_size)
  
  

In [ ]:
!echo "const unsigned char model[] = {" > ./content/model.h
!cat gesture_model.tflite | xxd -i      >> ./content/model.h
!echo "};"                              >> ./content/model.h

import os
model_h_size = os.path.getsize("./content/model.h")
print(f"Header file, model.h, is {model_h_size:,} bytes.")
print("\nOpen the side panel (refresh if needed). Double click model.h to download the file.")

In [ ]:
from sklearn.metrics import f1_score

# Assuming 'y_true' contains the true labels and 'y_pred' contains the predicted labels
f1 = f1_score(outputs_test, pre)